In [ ]:
using Pkg
if isfile("../Project.toml") && isfile("../Manifest.toml")
    Pkg.activate("..");
    ENV["PYTHON"] = "python3";
end

function printer(x::Any, verbose="")
    if verbose != ""
        println(verbose)
    end
    show(stdout, "text/plain", x)
    println("")
end

using Plots, Random, Distributions, LinearAlgebra, JuliaProbo
gr();

In [ ]:
function ch07_reset_mcl2()
    dt = 0.1
    # environment
    xlim = [-5.0, 5.0]
    ylim = [-5.0, 5.0]
    # id of landmark must start from 0 with 1 step
    landmarks = [Landmark([2.0, -3.0], 0), Landmark([3.0, 3.0], 1), Landmark([-4.0, 2.0], 2)]
    envmap = Map()
    push!(envmap, landmarks)
    world = World(xlim, ylim)
    push!(world, envmap)
    # robot side
    initial_pose = [0.0, 0.0, 0.0]
    estimator = ResetMcl(initial_pose, 100; xlim=xlim, ylim=ylim, α_threshold=0.005)
    circling_agent = EstimatorAgent(0.2, 10.0*pi/180, dt, estimator)
    robot = RealRobot(initial_pose, circling_agent, RealCamera(landmarks); color="red", expected_kidnap_time=30)
    push!(world, robot)
    anim = @animate for i in 1:600
        t = dt * i
        annota = "t = $(round(t, sigdigits=3))[s]"
        p = draw(world, annota)
        z = observations(robot.sensor_, robot.pose_; noise=true, bias=true)
        v, ω = decision(circling_agent, z, envmap)
        state_transition(robot, v, ω, dt; move_noise=true, vel_bias_noise=true, kidnap=true)
    end
    gif(anim, "images/ch07_reset_mcl2.gif", fps=10)
    return estimator
end

In [ ]:
pf = ch07_reset_mcl2();